In [117]:
import requests
import re
from tqdm import tqdm
import time
import string
import random

In [87]:
basepath = 'http://www.ermak.su/context/afor_posl_zag/poslov_{}.htm'
html_re = '''<p class="MsoNormal" style="text-indent: 40px; margin-left: 20px; margin-right: 10px; margin-top: 6px; margin-bottom: 6px">([А-Яа-я\.\,\?\!\-\s]+?)</p>'''
paths = [basepath.format(char) for char in list(string.ascii_lowercase)+['zh','sh']]

In [88]:
%%time
zavets = []
for path in tqdm(paths):
    try:
        r = requests.get(path)
        if r.status_code == 200:
            chunk = re.findall(html_re, r.text, re.DOTALL)
            zavets += chunk
        else: 
            print(path, r.status_code)
    except Exception as e:
        print(path, e)

 11%|████████▉                                                                          | 3/28 [00:00<00:00, 26.23it/s]

http://www.ermak.su/context/afor_posl_zag/poslov_f.htm 404


 32%|██████████████████████████▋                                                        | 9/28 [00:00<00:00, 25.65it/s]

http://www.ermak.su/context/afor_posl_zag/poslov_j.htm 404


 57%|██████████████████████████████████████████████▊                                   | 16/28 [00:00<00:00, 26.68it/s]

...

http://www.ermak.su/context/afor_posl_zag/poslov_w.htm 404


 82%|███████████████████████████████████████████████████████████████████▎              | 23/28 [00:00<00:00, 26.74it/s]

http://www.ermak.su/context/afor_posl_zag/poslov_x.htm 404


100%|██████████████████████████████████████████████████████████████████████████████████| 28/28 [00:01<00:00, 26.98it/s]


Wall time: 1.05 s


In [113]:
len(zavets)

1644

In [119]:
%%time
for i, p in enumerate(zavets):
    replacements = [
        ('\s?Ср\.\s?', ' '),
        ('\(.*?\)', ''),
        ('(\S) +(\.)', '\g<1>\g<2>'),
        ('[a-zA-Z\\t\\n\\r]*', ''),
        ('^\W+(\w+)', '\g<1>'),
        (' +', ' '),
    ]
    for old, new in replacements:
        p = re.sub(old, new, p)
    zavets[i] = p

Wall time: 84.2 ms


In [180]:
path2 = 'http://libbabr.com/?book=6972'
rt = requests.get(path2).text
html_re2 = '''<p>([А-Яа-я\,\.\!\?\s]{3,})\s\r<p>'''
zavets2 = [re.sub('^\s(.+)','\g<1>', z) for z in re.findall(html_re2, rt, re.DOTALL)]

In [183]:
len(zavets2)

150

In [191]:
zavets_all = list(set(zavets + zavets2))
len(zavets_all)

In [193]:
with open('zavets.txt', 'w', encoding='utf-8') as f:
    for i in zavets_all:
        f.write("{}\n".format(i))